This is a notebook that merges the parquet files with four datasets taken from https://dbr.abs.gov.au/region.html?lyr=ra&rgn=20 which are for major cities, inner regional, outer regional and remote australia inside victoria


In [3]:
# downloading datasets
import urllib.request as req

# url = [major city, inner regional, outer regional, remote]
url = ['https://dbr.abs.gov.au/processedData/csv/RA_20.csv', 'https://dbr.abs.gov.au/processedData/csv/RA_21.csv','https://dbr.abs.gov.au/processedData/csv/RA_22.csv','https://dbr.abs.gov.au/processedData/csv/RA_23.csv']
url_header = ['major', 'inner_r', 'outer_r', 'remote']
urls = {}
for i,j in enumerate(url_header):
    req.urlretrieve(url[i], '../data/raw/{}.csv'.format(j))

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/abs/major.csv'

In [1]:
# converting csv files to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("CSV to DataFrame").getOrCreate()


your 131072x1 screen size is bogus. expect trouble
24/10/01 14:34:43 WARN Utils: Your hostname, DESKTOP-RBVA59Q resolves to a loopback address: 127.0.1.1; using 172.26.88.196 instead (on interface eth0)
24/10/01 14:34:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/01 14:34:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
dfs = []
url_header = ['major', 'inner_r', 'outer_r', 'remote']
region = ['metropolitan victoria', 'inner regional victoria', 'outer regional victoria', 'remote victoria']

i = 1
while i < 5:
    df = spark.read.csv('../data/raw/{}.csv'.format(url_header[i-1]), header=True, inferSchema=True)
    
    # only using columns from 2018-2022
    df = df.drop('2011', '2015', '2016', '2023')
    # assigning region values
    df = df.withColumn('region', F.lit(region[i-1]))
    dfs.append(df)
    i += 1

In [22]:
from functools import reduce

merged_df = reduce(lambda df1, df2: df1.union(df2), dfs)

In [123]:
merged_df.write.parquet('../data/raw/region_summary.parquet')



AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/ulizuli/real_estate_data/real estate data/project-2-group-real-estate-industry-project-22/data/raw/region_summary.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [124]:
import os
# deleting unused files
files = ['../data/raw/major.csv', '../data/raw/outer_r.csv', '../data/raw/inner_r.csv', '../data/raw/remote.csv']

for file in files:
    if os.path.exists(file):
        os.remove(file)
        print(f"{file} has been deleted.")
    else:
        print(f"{file} does not exist.")

../data/raw/major.csv has been deleted.
../data/raw/outer_r.csv has been deleted.
../data/raw/inner_r.csv has been deleted.
../data/raw/remote.csv has been deleted.


=======================================================================================================================\
changing postcode dataset to suit what we want

we do this to merge the data into our property dataset by linking our region names with the remoteness area names by using the australian postcode dataset by matthew proctor

In [6]:
df = spark.read.csv('../data/raw/australian_postcodes.csv', header=True, inferSchema=True)

In [29]:
df.show()

+-----+--------+--------------------+-----+-----------+------------+--------------------+-----------------+--------------------+-----+--------------+----+--------------------+------+-----------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------+-------+-------+------------+--------+--------+----+----------------+----------+--------+------------------+--------------------+-------+----------+------------------+--------+-----------+
|   id|postcode|            locality|state|       long|         lat|                  dc|             type|              status|  sa3|       sa3name| sa4|             sa4name|region|Lat_precise|Long_precise|SA1_CODE_2021|SA1_NAME_2021|SA2_CODE_2021|SA2_NAME_2021|SA3_CODE_2021|SA3_NAME_2021|SA4_CODE_2021|SA4_NAME_2021|RA_2011|RA_2016|RA_2021|RA_2021_NAME|MMM_2015|MMM_2019| ced|        altitude|chargezone|phn_code|          phn_name|           lgaregion|lgacode|electorate|  elector

In [7]:
# only want postcode, locality, state and ra_2021_name columns
wanted_col = ['postcode', 'locality', 'state', 'lgaregion', 'RA_2021_NAME']
new_df = df.select(*wanted_col)
# only take rows from victoria
new_df = new_df.filter(new_df['state'] == 'VIC')
# remove the state column as its not important anymore
new_df = new_df.drop('state')

In [75]:
new_df.show()

+--------+--------------------+------------+--------------------+
|postcode|            locality|   lgaregion|        RA_2021_NAME|
+--------+--------------------+------------+--------------------+
|    3000|           MELBOURNE|   Melbourne|Major Cities of A...|
|    3001|           MELBOURNE|   Melbourne|                NULL|
|    3002|      EAST MELBOURNE|   Melbourne|Major Cities of A...|
|    3003|      WEST MELBOURNE|   Melbourne|Major Cities of A...|
|    3004|           MELBOURNE|   Melbourne|Major Cities of A...|
|    3004|ST KILDA ROAD CEN...|   Melbourne|Major Cities of A...|
|    3004|ST KILDA ROAD MEL...|   Melbourne|Major Cities of A...|
|    3005|  WORLD TRADE CENTRE|   Melbourne|                NULL|
|    3006|         SOUTH WHARF|Port Phillip|Major Cities of A...|
|    3006|           SOUTHBANK|Port Phillip|Major Cities of A...|
|    3008|           DOCKLANDS|   Melbourne|Major Cities of A...|
|    3010|UNIVERSITY OF MEL...|   Melbourne|                NULL|
|    3011|

In [104]:
df1 = spark.read.csv('../data/raw/census_population.csv', header=True, inferSchema=True)
df1.show()

+-------------+-------+-------+-------+------------+------------+------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+----------+----------+----------+---------------------------+---------------------------+---------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------------+------------------+--------

In [105]:
df1 = df1\
    .withColumnRenamed('POA_CODE_2021', 'postcode') \
    .withColumnRenamed('Tot_P_M', 'total male population - 2021') \
    .withColumnRenamed('Tot_P_F', 'total female population - 2021') \
    .withColumnRenamed('Tot_P_P', 'total population - 2021') \
    .withColumnRenamed('Australian_citizen_P', 'australian citizens')
df1.show()

+--------+----------------------------+------------------------------+-----------------------+------------+------------+------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+----------+----------+----------+---------------------------+---------------------------+---------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+---------------------

In [106]:
cols = ['postcode', 'total male population - 2021', 'total female population - 2021', 'total population - 2021', 'australian citizens']
df1 = df1.withColumn("postcode", F.col('postcode').substr(4, 4))
df1 = df1.select(*cols)
df1.show()

+--------+----------------------------+------------------------------+-----------------------+-------------------+
|postcode|total male population - 2021|total female population - 2021|total population - 2021|australian citizens|
+--------+----------------------------+------------------------------+-----------------------+-------------------+
|    3000|                       21548|                         21539|                  43084|              14713|
|    3002|                        2353|                          2545|                   4896|               4045|
|    3003|                        4093|                          3926|                   8025|               4366|
|    3004|                        5655|                          5827|                  11482|               7103|
|    3006|                       11499|                         11197|                  22699|              11460|
|    3008|                        8037|                          7458|          

In [107]:
merged_df = new_df.join(df1, on = 'postcode', how = 'left')
merged_df.show()

+--------+--------------------+------------+--------------------+----------------------------+------------------------------+-----------------------+-------------------+
|postcode|            locality|   lgaregion|        RA_2021_NAME|total male population - 2021|total female population - 2021|total population - 2021|australian citizens|
+--------+--------------------+------------+--------------------+----------------------------+------------------------------+-----------------------+-------------------+
|    3000|           MELBOURNE|   Melbourne|Major Cities of A...|                       21548|                         21539|                  43084|              14713|
|    3001|           MELBOURNE|   Melbourne|                NULL|                        NULL|                          NULL|                   NULL|               NULL|
|    3002|      EAST MELBOURNE|   Melbourne|Major Cities of A...|                        2353|                          2545|                   4896| 

In [108]:
merged_df.write.mode('overwrite').parquet('../data/raw/victorian_postcodes.parquet')

In [109]:
region_df = spark.read.parquet("../data/raw/region_summary.parquet")

In [110]:
region_df.show()

+------------+--------------------+--------------------+---------+---------+---------+---------+---------+--------------------+
|Measure Code|  Parent Description|         Description|     2018|     2019|     2020|     2021|     2022|              region|
+------------+--------------------+--------------------+---------+---------+---------+---------+---------+--------------------+
|    ERP_P_20|Estimated residen...|Estimated residen...|5009197.0|5102070.0|5159031.0|5075096.0|5141424.0|metropolitan vict...|
|      ERP_21|Estimated residen...|Population densit...|    990.9|   1009.3|   1020.5|   1003.9|   1017.1|metropolitan vict...|
|    ERP_M_20|Estimated residen...|Estimated residen...|2475909.0|2523950.0|2551649.0|2508441.0|2538968.0|metropolitan vict...|
|    ERP_F_20|Estimated residen...|Estimated residen...|2533288.0|2578120.0|2607382.0|2566655.0|2598457.0|metropolitan vict...|
|      ERP_19|Estimated residen...|Median age - male...|     34.8|     34.9|     35.3|     35.9|     35.

In [111]:
region_df = region_df.withColumn(
    'region',
    F.when(F.col('region') == 'metropolitan victoria', 'Major Cities of Australia' )
    .otherwise(F.col('region'))).withColumn(
    'region',
    F.when(F.col('region') == 'inner regional victoria', 'Inner Regional Australia' )
    .otherwise(F.col('region'))
    ).withColumn(
    'region',
    F.when(F.col('region') == 'outer regional victoria', 'Outer Regional Australia' )
    .otherwise(F.col('region'))
    ).withColumn(
    'region',
    F.when(F.col('region') == 'remote victoria', 'Remote Australia' )
    .otherwise(F.col('region'))
    )
    

In [112]:
region_df.show()


+------------+--------------------+--------------------+---------+---------+---------+---------+---------+--------------------+
|Measure Code|  Parent Description|         Description|     2018|     2019|     2020|     2021|     2022|              region|
+------------+--------------------+--------------------+---------+---------+---------+---------+---------+--------------------+
|    ERP_P_20|Estimated residen...|Estimated residen...|5009197.0|5102070.0|5159031.0|5075096.0|5141424.0|Major Cities of A...|
|      ERP_21|Estimated residen...|Population densit...|    990.9|   1009.3|   1020.5|   1003.9|   1017.1|Major Cities of A...|
|    ERP_M_20|Estimated residen...|Estimated residen...|2475909.0|2523950.0|2551649.0|2508441.0|2538968.0|Major Cities of A...|
|    ERP_F_20|Estimated residen...|Estimated residen...|2533288.0|2578120.0|2607382.0|2566655.0|2598457.0|Major Cities of A...|
|      ERP_19|Estimated residen...|Median age - male...|     34.8|     34.9|     35.3|     35.9|     35.

In [113]:
totalm_df = region_df.where(F.col('Measure Code') == 'ERP_M_20')
totalf_df = region_df.where(F.col('Measure Code') == 'ERP_F_20') 

In [114]:
totalm_df = totalm_df.withColumnRenamed('2022', 'total region male population - 2022')
totalf_df = totalf_df.withColumnRenamed('2022', 'total region female population - 2022')

In [115]:
col1 = ['region', 'total region male population - 2022']
col2 = ['region', 'total region female population - 2022']

totalm_df = totalm_df.select(*col1)
totalf_df = totalf_df.select(*col2)
totalm_df.show()

+--------------------+-----------------------------------+
|              region|total region male population - 2022|
+--------------------+-----------------------------------+
|Major Cities of A...|                          2538968.0|
|Inner Regional Au...|                           606425.0|
|Outer Regional Au...|                           128873.0|
|    Remote Australia|                             1744.0|
+--------------------+-----------------------------------+



In [116]:
merged_df = merged_df.withColumnRenamed('RA_2021_NAME', 'region')
merged_df1 = merged_df.join(totalm_df, on = 'region', how = 'left')
merged_df1 = merged_df1.join(totalf_df, on = 'region', how = 'left')

In [117]:
merged_df1 = merged_df1.drop('locality')
merged_df1.show()

+--------------------+--------+------------+----------------------------+------------------------------+-----------------------+-------------------+-----------------------------------+-------------------------------------+
|              region|postcode|   lgaregion|total male population - 2021|total female population - 2021|total population - 2021|australian citizens|total region male population - 2022|total region female population - 2022|
+--------------------+--------+------------+----------------------------+------------------------------+-----------------------+-------------------+-----------------------------------+-------------------------------------+
|Major Cities of A...|    3000|   Melbourne|                       21548|                         21539|                  43084|              14713|                          2538968.0|                            2598457.0|
|                NULL|    3001|   Melbourne|                        NULL|                          NULL|    

In [118]:
merged_df1.write.mode('overwrite').parquet('../data/raw/victorian_postcodes.parquet')

=====================================================================================\
now we are merging our population data with our property data

In [119]:
# reading dataframes
property_df = spark.read.parquet('../data/raw/property_data.parquet')
pop_df = spark.read.parquet('../data/raw/victorian_postcodes.parquet')

In [120]:
property_df.show()

+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+--------------------+
|                 url|postcode|         suburb|                name|           cost_text|beds|baths|parking|       property_type|
+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+--------------------+
|https://www.domai...|    3141|    south-yarra|(Leased) 3 Yarra ...|                $460|   1|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3125|        burwood|04/390 Burwood Hi...|$310 per week, wi...|   1|    1|      0|Apartment / Unit ...|
|https://www.domai...|    3156| ferntree-gully|1 & 2/32 Folkston...|                $800|   4|    3|      1|               House|
|https://www.domai...|    3228|        torquay|1 Acacia Street, ...|     $1,000 per week|   4|    2|      2|               House|
|https://www.domai...|    3200|frankston-north|1 Aleppo Crescent...|                $450| 

In [121]:
# merge on postcode
final_merge_df = property_df.join(pop_df, on = 'postcode', how = 'left')

In [122]:
final_merge_df.show()

+--------+--------------------+---------------+--------------------+--------------------+----+-----+-------+--------------------+--------------------+-----------+----------------------------+------------------------------+-----------------------+-------------------+-----------------------------------+-------------------------------------+
|postcode|                 url|         suburb|                name|           cost_text|beds|baths|parking|       property_type|              region|  lgaregion|total male population - 2021|total female population - 2021|total population - 2021|australian citizens|total region male population - 2022|total region female population - 2022|
+--------+--------------------+---------------+--------------------+--------------------+----+-----+-------+--------------------+--------------------+-----------+----------------------------+------------------------------+-----------------------+-------------------+-----------------------------------+----------------

In [125]:
final_merge_df.write.mode('overwrite').parquet('../data/curated/merged_df.parquet')